In [9]:
using LinearAlgebra
using SparseArrays
using Plots

In [41]:
function xy(
    x0::Float64, 
    y0::Float64, 
    L1::Float64, 
    L2::Float64, 
    noelms1::Int64, 
    noelms2::Int64
)::Tuple{Vector{Float64}, Vector{Float64}}
    VX = repeat(collect(LinRange(x0, x0+L1, noelms1+1)), inner=noelms2+1)
    VY = repeat(collect(LinRange(y0+L2, y0, noelms2+1)), noelms1+1)
    return VX, VY
end

function conelmtab(noelms1::Int64, noelms2::Int64)::Matrix{Int64}
    k = [i for i in 1:(noelms1*(noelms2+1)) if i % (noelms2+1) != 0]

    return [
        k (2 + noelms2 .+ k) (1 + noelms2 .+ k);
        k (1 .+ k) (2 + noelms2 .+ k)
    ]
end

function basfun(
    VX::Vector{Float64}, 
    VY::Vector{Float64}, 
    EToV::Matrix{Int64}
)::Tuple{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}}
    xjs = VX[EToV[:, [2,3,1]]]
    yjs = VY[EToV[:, [2,3,1]]]

    xks = VX[EToV[:, [3,1,2]]]
    yks = VY[EToV[:, [3,1,2]]]

    as = xjs .* yks - xks .* yjs
    bs = yjs - yks
    cs = xks - xjs

    return as, bs, cs
end

function constructBnodes(
    VX::Vector{Float64},
    VY::Vector{Float64},
    tol::Float64,
    fd::Function,
)::Vector{Int64}
    return findall(abs.(fd.(VX, VY)) .<= tol)
end

constructBnodes (generic function with 1 method)

# Exercise 5.1

In [42]:
function assembly(
    VX::Vector{Float64},
    VY::Vector{Float64},
    EToV::Matrix{Int64},
    lam1::Float64,
    lam2::Float64,
    qt::Vector{Float64},
    c::Vector{Float64}
)::Tuple{SparseMatrixCSC{Float64, Int64}, SparseMatrixCSC{Float64, Int64}, Vector{Float64}}
    N = size(EToV)[1]
    M = length(VX)

    R = spzeros(M, M)
    S = spzeros(M, M)
    b = zeros(M)

    ats, bts, cts = basfun(VX, VY, EToV)
    deltas = abs.(sum(ats, dims=2) ./ 2)
    qs = deltas .* sum(qt[EToV], dims=2) / 9
    cs = sum(c[EToV], dims=2) / 3
    c_matrix = Diagonal(fill(1/12, 3)) + fill(1/12,3,3) # specific heat matrix for computation

    for n in 1:N
        delta = deltas[n]
        cn_matrix = c_matrix .* delta .* cs[n]
        q = qs[n]
        bt = bts[n, :]
        ct = cts[n, :]

        for r in 1:3
            i = EToV[n,r]
            b[i] += q

            for s in r:3
                j = EToV[n,s]
                kn = (lam1*bt[r]*bt[s] + lam2*ct[r]*ct[s]) / (4 * delta)
                R[min(i, j), max(i, j)] += kn
                S[min(i, j), max(i ,j)] += cn_matrix[r,s]
            end
        end
    end

    return R, S, b
end

assembly (generic function with 1 method)

In [46]:
function dirbc(
    bnodes::Vector{Int64}, 
    f::Vector{Float64}, 
    R::SparseMatrixCSC{Float64, Int64}, 
    S::SparseMatrixCSC{Float64, Int64}, 
    b::Vector{Float64}
)::Tuple{SparseMatrixCSC{Float64, Int64}, SparseMatrixCSC{Float64, Int64}, Vector{Float64}, Vector{Float64}}
    d = zeros(length(b))
    for (i, k) in enumerate(bnodes)
        b[k] = 0

        indices1R = findall(R[1:k-1, k] .!= 0) 
        indices2R = findall(R[k, 1+k:end] .!= 0) .+ k
        indices1S = findall(S[1:k-1, k] .!= 0) 
        indices2S = findall(S[k, 1+k:end] .!= 0) .+ k

        temp1 = R[indices1R,k] .* f[i]
        temp2 = R[k,indices2R] .* f[i]

        b[indices1R] -= temp1
        b[indices2R] -= temp2

        d[indices1R] += temp1
        d[indices2R] += temp2

        R[indices1R, k] .= 0
        R[k, indices2R] .= 0
        S[indices1S, k] .= 0
        S[k, indices2S] .= 0
        
        S[k, k] = 1
    end

    return R, S, b, d
end

dirbc (generic function with 1 method)

In [45]:
ct(x,y) = 1.0
qt(x,y) = 0.0
f(x,y) = 1.0
distD(x, y) = min(x-x0, y-y0,x0+L1-x, y0+L2-y)

lam1 = lam2 = nu = 2.0
noelms1 = 4
noelms2 = 3
x0 = 0.0
y0 = 0.0
L1 = 1.0
L2 = 1.0
tol = 0.001

VX, VY = xy(x0, y0, L1, L2, noelms1, noelms2)
EToV = conelmtab(noelms1, noelms2)
bnodes = constructBnodes(VX, VY, tol, distD)

R, S, b = assembly(VX, VY, EToV, lam1, lam2, qt.(VX, VY), ct.(VX,VY))
R, S, b, d = dirbc(bnodes,f.(VX,VY),R,S,b)

display(spy(R,markersize=5))
display(spy(S,markersize=5))
display(b)
display(d)

LoadError: MethodError: [0mCannot `convert` an object of type 
[0m  Tuple{SparseMatrixCSC{Float64,Int64},[92mVector{Float64}[39m}[0m to an object of type 
[0m  Tuple{SparseMatrixCSC{Float64,Int64},[91mSparseMatrixCSC{Float64, Int64}[39m,[91mVector{Float64}[39m}

[0mClosest candidates are:
[0m  convert(::Type{T}, ::Tuple{Vararg{Any, N}}) where {N, T<:Tuple}
[0m[90m   @[39m [90mBase[39m [90m[4messentials.jl:452[24m[39m
[0m  convert(::Type{T}, [91m::T[39m) where T<:Tuple
[0m[90m   @[39m [90mBase[39m [90m[4messentials.jl:451[24m[39m
[0m  convert(::Type{T}, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mBase.jl:84[24m[39m
[0m  ...
